# Grading process


The submission notebook will be autovalidated with `papermill`. The exact command is the following:

```bash
papermill <notebook-name>.ipynb <notebook-name>-run.ipynb .ipynb -p TEST True
```

Papermill will inject new cell after each cell tagged as `parameters` (see `View > Cell toolbar > Tags`). Notebook will be executed from top to bottom in a linear order. `solutions.py` contains correct implementations used to validate your solutions.

Please, **fill `STUDENT` variable with the name of submitting student**, so that we can collect the results automatically. Please, **do not change `TEST` variable** and `validation` cells. If you need to inject your own code for testing, wrap it into

```python
if not TEST:
    ...
```

Different problems give different number of points. All problems in the basic section give 1 point, while all problems in intermediate section give 2 points.

Each problem contains specific validation details. You need to fill each cell tagged `solution` with your code. Note, that solution function must self-contained, i.e. it must not use any state from the notebook itself.

# Dataset

All problems in the assignment use [electricity load dataset](https://archive.ics.uci.edu/ml/datasets/ElectricityLoadDiagrams20112014). Some functions/methods accept data itself, and in that case it's a Pandas dataframe as obtained by

```python
df = pd.read_csv("LD2011_2014.txt",
                 parse_dates=[0],
                 delimiter=";",
                 decimal=",")
df.rename({"Unnamed: 0": "timestamp"}, axis=1, inplace=True)
```

In contrast, whenever a function/method accepts a filename, it's the filename of **unzipped** data file (i.e. `LD2011_2014.txt`). When testing, do not rely on any specific location of the dataset, as validation environment will most certainly different from your local one. Hence, calls like

```python
df = pd.read_csv("<your-local-directory>/LD2011_2014.txt")
```

will fail.

In [ ]:
import numpy as np
import pandas as pd
import torch
import random

from datetime import timedelta

In [ ]:
STUDENT = "Omri Kazelnik"

In [ ]:
ASSIGNMENT = 1
TEST = False

In [ ]:
if TEST:
    import solutions
    total_grade = 0
    MAX_POINTS = 12

# Pandas

### 1. Resample the dataset (1 point)

Resample the dataset to 1-hour resolution. Use `mean` as an aggregation function. Your function must output a dataframe, with the same structure as the original one (i.e. not indexed by datetime).

In [ ]:
def el_resample(df):
    resampled_df = df.resample('1H', on='timestamp').mean()
    resampled_df['timestamp'] = resampled_df.index
    resampled_df.reset_index(drop=True, inplace=True)
    return resampled_df

In [ ]:
PROBLEM_ID = 1

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, el_resample)

### 2. Consumption peaks (1 point)

For each household, calculate, which month in 2014 had the highest consumption. Your function must output series, indexed by household ID (e.g., `MT_XXX`), and containing month as an integer (`1-12`).

In [ ]:
def cons_peak(df):
    peak_df = df.copy()
    peak_df['timestamp'] = pd.to_datetime(peak_df['timestamp'])
    peak_df = peak_df[peak_df['timestamp'].dt.year == 2014]
    peak_df = peak_df.resample('1M', on='timestamp').mean()
    return peak_df.idxmax().dt.month

In [ ]:
PROBLEM_ID = 2

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, cons_peak)

# PyTorch

### 3. Find minimum (2 points)

Consider the following scalar function:

$$
f(x) = ax^2 + bx + c
$$

Given $a,b,c$, find $x$, which minimizes $f(x)$, and minimum value of $f(x)$. Note this:

- $a,b,c$ are fixed, and generated in such a way, that minimum always exists ($f(x)$ is convex),
- $x$ is a scalar value, i.e. 0-dimensional tensor.

For reference, see `generate_coef` function, which is used to generate coefficients. Note, that since optimization process is not completely deterministic, the output is considered correct, if it falls within `1e-3` of actual values.

This problem must be solved as an optimization one using gradient descent.

For that, use only PyTorch functionality, `SciPy` (or alike) optimization routines are not allowed, neither is direct calculation using coefficients.

In [ ]:
def generate_coeffs():
    a = torch.rand(size=()) * 10
    b = -10 + torch.rand(size=()) * 10
    c = -10 + torch.rand(size=()) * 10
    return a, b, c

def func(x, a, b, c):
    return x.pow(2) * a + x * b + c

In [ ]:
def find_min(a, b, c):
    dtype = torch.float
    device = torch.device("cpu")

    x = torch.randint(-1, 1, (1,), dtype=dtype, device=device, requires_grad=True)
    alpha = 1e-2
    stop_precision = 1e-7

    while x.grad is None:
        y = func(x=x, a=a, b=b, c=c)
        y.backward()
        update = x.data - alpha * x.grad.data
        if torch.abs(x - update) > stop_precision:
            x.grad = None
        x.data = update

    return x.data[0], func(x=x.data, a=a, b=b, c=c)

In [ ]:
PROBLEM_ID = 3

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, find_min)

### 4. PyTorch `Dataset` (3 points)

Implement a `torch.utils.data.Dataset` sub-class for the electricity consumption data. Individual training instances must be week-long univarite series of hourly consumption (input, 168 values), followed by 24-hours long series of hourly consumption (output, 24 values) for a single household. Such a class can be used when training a consumption forecast model, which uses 7 days of historical consumption to forecast next 24 hours of consumption.

`__getitem__(self, idx)` must return a tuple of 1D tensors, `in_data` and `out_data`. `in_data` contains 168 hours of consumption (hourly), starting from some `start_ts`, while `out_data` must contain 24 hourly consumption values starting from `start_ts + 168 hours` for some household. `start_ts` should be sampled randomly.

Also, you need to implement a `get_mapping(self, idx)` method, which allows to calculate `(household, starting time) -> idx` correspondence.

This class will be validated as the following:

- dataset object is created with some random `samples`: `dataset = ElDataset(df, samples)` ,
- validator fetches random `idx` (between `0` and `len(dataset)`) from the dataset:
```python
household, start_ts = dataset.get_mapping(idx)
hist_data, future_data = dataset[idx]
```
- then, `hist_data` and `future_data` are compared with the data, obtained directly from `df` using `household, start_ts`.

In [ ]:
class ElDataset(torch.utils.data.Dataset):
    """Electricity dataset."""

    def __init__(self, df, samples):
        """
        Args:
            df: original electricity data (see HW intro for details).
            samples (int): number of sample to take per household.
        """
        self.raw_data = df.set_index("timestamp")
        self.raw_data = self.raw_data.resample('1H').mean()
        self.samples = samples
        self.min_date = None
        self.max_date = None
        self.mappings = None

        # setters
        self.set_min_max_dates()
        self.set_mappings()

    @staticmethod
    def create_household_from_id(household_id):
        if 0 <= household_id < 10:
            return f"MT_00{household_id}"
        elif 10 <= household_id <= 99:
            return f"MT_0{household_id}"
        else:
            return f"MT_{household_id}"

    def set_min_max_dates(self):
        self.min_date = min(self.raw_data.index).date()
        self.max_date = max(self.raw_data.index).date() - timedelta(days=8)

    def set_mappings(self):
        dates_pool = pd.date_range(start=self.min_date, end=self.max_date, freq='1H').tolist()
        self.mappings = {
            idx: pd.to_datetime(random.choice(dates_pool))
            for idx in range(self.__len__())
        }

    def __len__(self):
        return self.samples * self.raw_data.shape[1]

    def __getitem__(self, idx):
        household, start_training_ts = self.get_mapping(idx=idx)
        end_training_ts = pd.to_datetime(start_training_ts + timedelta(hours=167))  # takes 7 days for train
        start_test_ts = pd.to_datetime(end_training_ts + timedelta(hours=1))
        end_test_ts = pd.to_datetime(start_test_ts + timedelta(hours=23))  # takes 24 hours for test

        return torch.tensor(self.raw_data.loc[start_training_ts:end_training_ts][household].values), \
           torch.tensor(self.raw_data.loc[start_test_ts:end_test_ts][household].values)

    def get_mapping(self, idx):
        household_id = (idx // self.samples) + 1
        return ElDataset.create_household_from_id(household_id), self.mappings[idx]

In [ ]:
PROBLEM_ID = 4

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, ElDataset)

# Your grade

In [ ]:
if TEST:
    print(f"{STUDENT}: {total_grade}")